<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/AttaccoScanner/Attacco_scanner_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

##Import ART

In [1]:
#importa ART
!pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 7.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 26.3 MB 1.5 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


## Import librerie

In [2]:
#Librerie

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
#import time
#import os
#import shutil
#import copy

import pandas as pd
import matplotlib.pyplot as plt
import sys


##Check device


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

1.9.0+cu111
True
Tesla P100-PCIE-16GB
cuda:0


##Def path

In [4]:
#scanner
scanner_name = 'HiScan'

#salvataggio modello
model_name = 'VGG19_paper_10epoc_lr5_bs200_adam'
path_model = F"/content/gdrive/My Drive/ModelliCNN/Scanner/{scanner_name}/{model_name}" 

#dataset
pathTestset=F'/content/gdrive/MyDrive/Dataset_impronte/test/{scanner_name}'

##Collegamento google drive

In [5]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Caricamento dataset

In [6]:
test_dataset = datasets.ImageFolder(pathTestset,transforms.ToTensor())
dim_set=len(test_dataset)
print(dim_set)

2500


##Dataloader

In [7]:
testgen=torch.utils.data.DataLoader(test_dataset, pin_memory=True, batch_size=1,num_workers=2)
print(len(testgen))

2500


##Nomi classi

In [8]:
classes_name=test_dataset.classes
class_number=len(classes_name)
print(classes_name)
print(class_number)

['Live', 'Spoof']
2


#**Dataset attack**

##Caricamento dataset

In [9]:
data_transform_attack=data_transform=transforms.Compose([transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
test_dataset_at = datasets.ImageFolder(pathTestset,transform=data_transform_attack)

##Dataloader

In [10]:
testgen_at=torch.utils.data.DataLoader(test_dataset_at, pin_memory=True, batch_size=1,num_workers=2)

#**Riduzione dataset**

##riduzione dataset originale

In [11]:
test_dataset.samples=test_dataset.samples[:5]
test_dataset.targets=test_dataset.targets[:5]

dim_set=len(test_dataset)
print(dim_set)

5


##riduzione dataset attack

In [12]:
test_dataset_at.samples=test_dataset_at.samples[:5]
test_dataset_at.targets=test_dataset_at.targets[:5]

# **Creazione modello**

> non ho bloccato i parametri inferiori e sbloccato quelli del classificatore, vedi se funziona



##Load model pre-trained

In [11]:
model = models.vgg19(pretrained=True,progress=True)
print(model)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

##Aggiunta classificatore

In [12]:
model.classifier[6]=nn.Linear(4096,2) #per vgg19
#model.classifier=nn.Linear(1920,2) #per densenet201

print(model) 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

##Load pesi

In [13]:
model.load_state_dict(torch.load(path_model))#,map_location=torch.device('cpu')))


<All keys matched successfully>

##Set iperparametri

In [14]:
learning_rate = 1e-5

criterion = nn.CrossEntropyLoss() #criterio dell'aggiornamento del gradiente: minimizzazione funzione loss entropia
optimizer = optim.Adam(model.classifier.parameters(),lr=learning_rate)#,momentum=momentum)

# **Def funzioni**

##Calcolo perturbazioni



In [15]:
def compute_perturb(x,x_adv):
  '''
    x: img originali
    x_adv: img contraddittorie
    return: pertubazioni, media, perturbazioni normalizzate in [0,1] 
  '''
  perturb=x_adv-x
  m=np.mean(np.abs((x_adv - x)))
  print('Average perturbation: {:4.2f}'.format(m))
  min, max = np.amin(perturb), np.amax(perturb)
  perturb_norm = (perturb - min) / (max - min)
  return perturb,m,perturb_norm

##Testing

senza media

In [16]:
def testing(x,y,classifier):
  '''
    x: img test
    y: classi reali
    classifier: modello di tipo Keras addestrato
    return: valori probabilità, classi predette, accuracy
  '''
  
  value_preds=classifier.predict(x) #contiene i valori tra 0 e 1 predetti per ognuna delle 10 classi e per ogni immagine
  preds = np.argmax(value_preds, axis=1) #(le predizioni vanno da 0 a 9 e indicano la classe predetta)
  acc = (np.sum(preds == np.argmax(y, axis=1)) / y.shape[0])*100
  print("Accuracy on test set:", (round(acc,2)))
  #print("classi predette: ",preds)
  return value_preds,preds,acc

con media

In [17]:
#calcola la dimensione che devono avere i vari crop
def calc_size(n):
  '''
  n: int 
  return: 80% of n
  '''
  return tuple(int(np.ceil(i * (80/100))) for i in n)

def test_average(model,test_loader,):
  '''
  model: model trained
  test_loader: dataloader 
  return: dataframe{n_img,class_predicted,class_real}
  '''

  Test = pd.DataFrame()
  model.eval()
  model.cuda()
  data_transform_test= transforms.Compose([transforms.Resize([224,224]),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
          ])
  i=0
  for input,label in test_loader:
    i+=1
    n=input.shape
    n_mod=calc_size(n[2:4])
    crop_transform=transforms.TenCrop((n_mod[0],n_mod[1]))#.to(device)
    crops=crop_transform(input)
    live=0
    spoof=0
    sys.stdout.write("\rImage {0}/{1}".format(i,len(test_loader)))
    sys.stdout.flush()
    for crop in crops:
      crop=data_transform_test(crop).to(device)
      outputs = model(crop)
      live+=outputs[0][0]
      spoof+=outputs[0][1]
    live=live/10
    spoof=spoof/10
    predicted=np.argmax([live,spoof])
    Test = Test.append({'real': classes_name[int(label)] ,
                        'predicted': classes_name[predicted],
                        'value_pred_live': live.item(),
                        'value_pred_spoof': spoof.item()},ignore_index = True)

  true_label = Test.real.values
  predicted = Test.predicted.values
  accuracy=round((np.sum((true_label == predicted).astype(int)))/Test.shape[0],4)*100
  print("\nAccuracy: {0}".format(accuracy))
  print("Shape dataframe: {0}".format(Test.shape))  

  return Test

##Print subplot

In [18]:
def print_subplot(n_img,preds,value_preds_adv,x_test,y_test,x_test_adv,perturb_norm,normalize=True):
  '''
    n_img: prime n immagini da visualizzare
    preds: classi predette per img originali
    value_preds_adv: probabilità predette img contraddittorie
    x_test: img originali
    y_test: classi originali
    x_test_adv: img contraddittoria
    perturb_norm: perturbazioni normalizzate
  '''
  
  if normalize:
    min,max=np.min(x_test),np.max(x_test)
    x_test = (x_test - min) / (max - min)
    min,max=np.min(x_test_adv),np.max(x_test_adv)
    x_test_adv = (x_test_adv - min) / (max - min)

  #nel seguente ciclo for si crea un vettore delle classi predette ordinato per probabilità decrescente
  for i in range(n_img):#(x_test.shape[0]):
    value=value_preds_adv[i]*100
    value_sorted=sorted(value,reverse=True)
    classes=[]
    for j in range(value.size) :
      ind=np.where(value==value_sorted[j]) #restituisce l'indice in value del valore uguale a value_sorted[i], quindi è la classe
      classes.append(classes_name[ind[0][0]]) #classes è il vettore finale
    
    value_sorted=[ round(elem, 2) for elem in value_sorted ]

    #in seguito per ogni immagine del test set si stampa un subplot
    fig = plt.figure(figsize=[10,10])
    plt.subplots_adjust(wspace=0.9)
    print('\033[1m'+"IMMAGINE "+'\033[1m',i) #valore END: '\033[0m'
    #originale
    ax1 = fig.add_subplot(131) #subplot con 3 righe e due colonne
    ax1.axis('off')
    #ax1.imshow(cv2.rotate(x_test[i],cv2.cv2.ROTATE_90_CLOCKWISE))
    ax1.imshow(x_test[i].transpose(1,2,0))
    ax1.title.set_text("ORIGINALE\nclasse reale: "+classes_name[np.argmax(y_test[i])]+"\nclasse predetta: "+str(preds[i]))
    #perturbazione
    ax2 = fig.add_subplot(132)
    #ax2.imshow(cv2.rotate(perturb_norm[i],cv2.cv2.ROTATE_90_CLOCKWISE))
    ax2.imshow(perturb_norm[i].transpose(1,2,0))
    ax2.axis('off')
    ax2.title.set_text("PERTURBAZIONE")
    #perturbata
    ax3 = fig.add_subplot(133)
    #ax3.imshow(cv2.rotate(x_test_adv[i],cv2.cv2.ROTATE_90_CLOCKWISE))#,aspect='auto')
    ax3.imshow(x_test_adv[i].transpose(1,2,0))
    ax3.axis('off')
    ax3.title.set_text("PERTURBATA\nclassi predette: "+str(classes)+"\ncon valori: "+str(value_sorted))
    plt.show()

##Array to dataloader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

def array2dataloader(x,y):
  tensor_x = torch.Tensor(x) # transform to torch tensor
  tensor_y = torch.Tensor(np.argmax(y,axis=1))

  my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
  return DataLoader(my_dataset) 

# **Esecuzione**

##creazione matrici

In [ ]:
from art.utils import to_categorical
x_test=[]
y_test=[]

for input,label in testgen_at:
  #x_test.append(input.permute(0,2,3,1).numpy()[0,:,:,:])
  print(i)
  x_test.append(input.numpy()[0,:,:,:])
  y_test.append(to_categorical(label.numpy(),2)[0])
  
x_test=np.array(x_test)
y_test=np.array(y_test)

print(x_test.shape)
print(y_test.shape)

min max

In [22]:
min,max=np.min(x_test),np.max(x_test)
print(min,max)

-2.117904 2.64


##Normalizza matrice [0,1]

In [ ]:
x_test = (x_test - min) / (max - min)
min=0
max=1

##Creazione classificatore ART Pytorch



In [23]:
from art.estimators.classification import PyTorchClassifier

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min, max),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(224, 224, 3),
    nb_classes=class_number,
)

##Attacco

###Definizione attacco

In [24]:
from art.attacks.evasion import FastGradientMethod
# FGM
attack = FastGradientMethod(estimator=classifier, eps=0.1)

###Generazione adversarial samples



In [25]:
x_test_adv = attack.generate(x=x_test,y=y_test) #aggiunge una perturbazione alle immagini del test set
#N.B. si possono passare le classi reali del test set (con y=y_test) e in questo caso FGM calcolerà le perturbazioni in modo che il classificatore
#non predica queste classi. Mentre, se non passo y_test, le calcolerà in modo da non fargli predire la classe predetta normalmente (la quale può essere diversa da quella reale
#se il classificatore non è corretto)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


##Calcolo perturbazione

In [26]:
perturb,m,perturb_norm=compute_perturb(x_test,x_test_adv)

Average perturbation: 0.09


##Testing

> Trasforma i due testing in una funzione e richiamala due volte



###test img orig

In [27]:
Test_orig=test_average(model,testgen)

Image 2500/2500
Accuracy: 93.76
Shape dataframe: (2500, 4)


###Testing no media img originali

In [28]:
value_preds,preds,acc=testing(x_test,y_test,classifier)

Accuracy on test set: 89.84


###test img perturb

da matrice adv a dataloader

In [ ]:
dataloader_adv=array2dataloader(x_test_adv,y_test)

In [30]:
Test_perturb=test_average(model,dataloader_adv)

Image 2500/2500
Accuracy: 61.12
Shape dataframe: (2500, 4)


###Testing no media img perturbate 

In [31]:
value_preds_adv,preds_adv,acc_adv=testing(x_test_adv,y_test,classifier)

Accuracy on test set: 17.2


##Stampa subplot prime N immagini

In [ ]:
#print_subplot(10,preds,value_preds_adv,x_test,y_test,x_test_adv,perturb_norm)
print_subplot(3,Test_orig.predicted,Test_perturb.values[:,2:4],x_test,y_test,x_test_adv,perturb_norm)